In [8]:
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from time import time
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

sample = pd.read_csv('../data/preprocessed/concat.csv', index_col=0)
sample['preprocessed'] = sample['preprocessed'] + ' '
sample.dropna(axis=0, inplace=True)

In [7]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

n_features = 1000
n_components = 25
n_top_words = 20

def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(5, 5, figsize=(30, 38), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

print("Loading dataset...")
t0 = time()
data = sample['preprocessed'].tolist()
data_samples = data
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.9, min_df=10, max_features=n_features, stop_words='english', ngram_range=(1,2)
)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print(
    "Fitting LDA models with tf features, n_features=%d..."
    % (n_features),
)
lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=10,
    learning_method="online",
    learning_offset=45.0,
    learning_decay=0.5,
    random_state=0,
    n_jobs=-1
)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, n_top_words, "Topics in LDA model")

Loading dataset...
done in 0.028s.
Extracting tf features for LDA...
done in 56.096s.
Fitting LDA models with tf features, n_features=1000...
done in 47.115s.


In [3]:
# Define Search Param
search_params = {'n_components': [25], 'learning_decay': [.5], 'learning_offset':[40.0, 45.0, 50.0]}

# Init the Model
lda = LatentDirichletAllocation(
    max_iter=10,
    learning_method="online",
    random_state=0,
    n_jobs=-1
)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, verbose=3)

# Do the Grid Search
model.fit(tf)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END learning_decay=0.5, learning_offset=40.0, n_components=25;, score=-7911364.824 total time=  46.9s
[CV 2/5] END learning_decay=0.5, learning_offset=40.0, n_components=25;, score=-7659295.324 total time=  47.4s
[CV 3/5] END learning_decay=0.5, learning_offset=40.0, n_components=25;, score=-7969947.127 total time=  48.7s
[CV 4/5] END learning_decay=0.5, learning_offset=40.0, n_components=25;, score=-7982538.663 total time=  47.7s
[CV 5/5] END learning_decay=0.5, learning_offset=40.0, n_components=25;, score=-8074076.239 total time=  47.2s
[CV 1/5] END learning_decay=0.5, learning_offset=45.0, n_components=25;, score=-7911233.281 total time=  47.8s
[CV 2/5] END learning_decay=0.5, learning_offset=45.0, n_components=25;, score=-7658192.767 total time=  47.7s
[CV 3/5] END learning_decay=0.5, learning_offset=45.0, n_components=25;, score=-7969155.239 total time=  48.4s
[CV 4/5] END learning_decay=0.5, learning_offset=45.

GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 n_jobs=-1, random_state=0),
             param_grid={'learning_decay': [0.5],
                         'learning_offset': [40.0, 45.0, 50.0],
                         'n_components': [25]},
             verbose=3)

In [4]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(tf))

Best Model's Params:  {'learning_decay': 0.5, 'learning_offset': 45.0, 'n_components': 25}
Best Log Likelihood Score:  -7918462.9442712115
Model Perplexity:  549.6367186007102


In [1]:
topic_dict = {
    0: 'Misc1',
    1: 'Ampelregierung',
    2: 'Innenpolitik',
    3: 'Familie',
    4: 'Gender',
    5: 'Wahlen',
    6: 'Justiz',
    7: 'Medien',
    8: 'Impfung',
    9: 'Abspann',
    10: 'Live',
    11: 'Lokal',
    12: 'Befragung',
    13: 'Krieg',
    14: 'Angela Merkel',
    15: 'Fußball',
    16: 'Ukrainekonflikt',
    17: 'Wirtschaft',
    18: 'Schule',
    19: 'COVID-19 Maßnahmen',
    20: 'Interview',
    21: 'USA',
    22: 'Misc2',
    23: 'International',
    24: 'CDU/CSU',
}

In [8]:
best_lda_model = lda

In [62]:
df = pd.read_csv('../data/samples/sample300.csv', index_col=0)
df.dropna(subset=['preprocessed'], inplace=True)
df_vectorized = tf_vectorizer.transform(df['preprocessed'].to_list())
lda_output = pd.DataFrame(lda.transform(df_vectorized))
lda_output['dominant topic'] = np.argmax(lda_output.values, axis=1)
lda_output['id'] = df['id'].to_list()
lda_output.set_index('id')


test = df.merge(lda_output, how='outer', on='id')
test['dominant topic'] = test['dominant topic'].apply(lambda row: topic_dict[row])

test.rename(columns = topic_dict, inplace = True)

In [63]:
test

medium                                              title  \
0           Junge Freiheit  Alice Weidel: "Appell an alle Bundestagsabgeor...   
1           NachDenkSeiten  Internet-Zensur: Schäuble bekämpft das Recht a...   
2              DER SPIEGEL  NSA-Chef bei IT-Konferenz: Auftritt voller Zwi...   
3              DER SPIEGEL  Tea Time im Tesla Model 3: Wir drehen eine Run...   
4              DER SPIEGEL        Zwangsräumung statt Enteignung | SPIEGEL TV   
...                    ...                                                ...   
1495        Junge Freiheit          Spot: Deutsche Geschichte für junge Leser   
1496        NachDenkSeiten  Hartz-IV-Urteil: Angriffe auf das Existenzmini...   
1497  ZDFheute Nachrichten  Debatte über Lockdown-Verlängerung | Markus La...   
1498        NachDenkSeiten  Davos – Willkommen im Paralleluniversum der „E...   
1499        Junge Freiheit         JF-TV Direkt. Dieter Stein und Konrad Adam   

               id duration                                         transcript  \
0     CYud2Ii8KXc     2:51  [Musik] freude sie sind dritte kraft geworden ...   
1     ib4o5Bs2TkQ     8:22  [Musik] nachdenkseiten die kritische website i...   
2     6WN7M2G6L1w     1:31  auf der konferenz black hat in las vegas stell...   
3     BuiPc_QaPBY    10:42  [Musik] [Applaus] [Musik] panico heute ist tea...   
4     i9DBAcYelXU     6:59  gut und willkommen zu spiegel tv laut un sozia...   
...           ...      ...                                                ...   
1495  i5isR0hf7v0     2:52  woher kommen wir wer sind wir und wo gehen wir...   
1496  -Rccm0NlWTA    11:46  [Musik] nachdenkseiten die kritische website h...   
1497  _G56rchtBns    31:35  wenn ja wann ist hamburgs arbeit mit diesen be...   
1498  KvQTDUQtNxM     7:34  [Musik] nachdenkseiten die kritische website d...   
1499  iDCq8WE6yp8     7:23  meinungen hintergründe analysen js tv direkt v...   

                                           preprocessed     Misc1  \
0     freude dritt kraft afg fulminant ergebnis hinl...  0.000769   
1     kritisch website internetzensur schäuble bekäm...  0.000305   
2     konferenz black vegas stellen chef nsa general...  0.001429   
3     panico tea time denken testen amerikanisch aut...  0.127466   
4     willkommen un sozialpakt wohnen menschenrecht ...  0.000455   
...                                                 ...       ...   
1495  ereignisse vergangenheit liefern antworten fra...  0.000690   
1496  kritisch website urteil angriffe existenzminim...  0.000221   
1497  hamburgs arbeit beschränkungen bewegungsradius...  0.185514   
1498  kritisch website davos willkommen paralleluniv...  0.000388   
1499  meinungen hintergründe analysen js bundesparte...  0.054213   

      Ampelregierung  Innenpolitik   Familie  ...  Ukrainekonflikt  \
0           0.000769      0.264072  0.035662  ...         0.000769   
1           0.000305      0.138707  0.000305  ...         0.000305   
2           0.001429      0.793036  0.001429  ...         0.001429   
3           0.000328      0.000328  0.000328  ...         0.000328   
4           0.000455      0.000455  0.240554  ...         0.000455   
...              ...           ...       ...  ...              ...   
1495        0.000690      0.000690  0.000690  ...         0.000690   
1496        0.011098      0.163521  0.000221  ...         0.075768   
1497        0.000062      0.159594  0.000062  ...         0.003919   
1498        0.000388      0.099432  0.000388  ...         0.000388   
1499        0.000268      0.115830  0.000268  ...         0.000268   

      Wirtschaft    Schule  COVID-19 Maßnahmen  Interview       USA     Misc2  \
0       0.000769  0.000769            0.000769   0.055633  0.000769  0.000769   
1       0.031560  0.006677            0.000305   0.000305  0.000305  0.035233   
2       0.001429  0.001429            0.001429   0.001429  0.093347  0.001429   
3       0.645638  0.000328            0.000328   0.000328  0.027478  0.000328   

In [58]:
test['dominant topic'].value_counts()

Innenpolitik          148
Medien                138
Wahlen                128
COVID-19 Maßnahmen    106
Live                   98
Wirtschaft             92
Misc1                  89
Misc2                  82
Justiz                 82
Ukrainekonflikt        80
Krieg                  72
Interview              62
Fußball                60
Familie                58
Befragung              50
Ampelregierung         38
Impfung                30
USA                    21
Angela Merkel          17
Gender                 13
International          12
Abspann                12
Schule                  7
CDU/CSU                 4
Lokal                   1
Name: dominant topic, dtype: int64

In [59]:
test.to_csv('../data/labeled/labeled300.csv')

In [39]:
import joblib
joblib.dump(lda, '../data/lda_model.pkl')

['../data/lda_model.pkl']

In [42]:
temp1 = joblib.load('../data/lda_model.pkl')
temp1.get_params()

In [44]:
joblib.dump(tf_vectorizer, '../data/cv_model.pkl')

['../data/cv_model.pkl']

In [45]:
temp2 = joblib.load('../data/cv_model.pkl')
temp2.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.9,
 'max_features': 1000,
 'min_df': 10,
 'ngram_range': (1, 2),
 'preprocessor': None,
 'stop_words': 'english',
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [ ]:
df = pd.read_csv('../data/labeled/labeled300.csv')
df.iloc[0]